In [1]:
!pip install torch_geometric networkx
!pip install torch torchvision torchaudio
!pip install torch_geometric networkx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.4 MB/s eta 0:00:00


In [2]:
import networkx as nx
import torch
from torch_geometric.data import Data

# Create a small social graph
G = nx.Graph()
edges = [
    (0, 1), (1, 2), (2, 0),    # normal users community
    (3, 4), (4, 5), (5, 3)     # suspicious cluster
]
G.add_edges_from(edges)

# One-hot node features (6 nodes, 6-dimensional)
num_nodes = 6
features = torch.eye(num_nodes, dtype=torch.float)

# Node labels: first 3 normal (0), last 3 bots (1)
labels = torch.tensor([0, 0, 0, 1, 1, 1], dtype=torch.long)

# Convert to PyG Data
edge_index = torch.tensor(list(G.edges()), dtype=torch.long).t().contiguous()
# Make edges bidirectional for undirected graph
edge_index = torch.cat([edge_index, edge_index.flip(0)], dim=1)

data = Data(x=features, edge_index=edge_index, y=labels)

# Define train/test masks (e.g., train on 4 nodes, test on 2 nodes)
data.train_mask = torch.tensor([1, 1, 1, 1, 0, 0], dtype=torch.bool)
data.test_mask  = torch.tensor([0, 0, 0, 0, 1, 1], dtype=torch.bool)

# show output
print(data)
print("x:\n", data.x)
print("edge_index:\n", data.edge_index)
print("y:", data.y)
print("train_mask:", data.train_mask)
print("test_mask:", data.test_mask)


Data(x=[6, 6], edge_index=[2, 12], y=[6], train_mask=[6], test_mask=[6])
x:
 tensor([[1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 1.]])
edge_index:
 tensor([[0, 0, 1, 3, 3, 4, 1, 2, 2, 4, 5, 5],
        [1, 2, 2, 4, 5, 5, 0, 0, 1, 3, 3, 4]])
y: tensor([0, 0, 0, 1, 1, 1])
train_mask: tensor([ True,  True,  True,  True, False, False])
test_mask: tensor([False, False, False, False,  True,  True])
